In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import re
from datetime import datetime
from supabase import create_client, Client

In [45]:
pd.set_option('display.max_rows',None)


In [46]:
pd.set_option('display.max_colwidth', None)

In [2]:
options = Options() 
driver = webdriver.Chrome(options=options)
driver.get('https://www.clasificadosonline.com/RealEstate.asp')

In [49]:
multi_check_box_toggle = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[1]')
multi_check_box_toggle.click()
time.sleep(2)

In [50]:
select_all = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[2]/div[1]/input')
select_all.click()
time.sleep(2)

In [51]:
time.sleep(10)
ver_listado = driver.find_element(By.XPATH,'//*[@id="BtnSearchListing"]')
ver_listado.click()

In [54]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')

In [ ]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')
order_by.click()
ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
ultimos_publicados.click()

In [43]:
def extract_property_id(link):
    match = re.search(r"ID=(\d+)", link)
    if match:
        return match.group(1)
    return None

In [44]:
def clean_real_estate_data(df):
    # Clean the 'Type' column by removing leading commas and whitespace
    df['type'] = df['type'].str.replace(r'^,\s*', '', regex=True).str.strip()
    
    # Clean the 'Broker' column using the combined logic
    def clean_broker(broker_name, barrio_name):
        try:

            # Remove the "ClasificadosOnline" text from the broker name
            broker_name = broker_name.replace("ClasificadosOnline", "").strip()

            # Extract the barrio name (e.g., "Las Mansiones de Villa Rica")
            barrio_cleaned = re.sub(r"[-]", " ", barrio_name.split("-", 1)[1])  # Clean barrio name

            # Remove the barrio part from the broker column
            broker_name_cleaned = re.sub(re.escape(barrio_cleaned), "", broker_name, flags=re.IGNORECASE)

            # Strip leading/trailing whitespace and "de" artifacts
            broker_name_cleaned = broker_name_cleaned.strip()
            if broker_name_cleaned.lower().startswith("de "):
                broker_name_cleaned = broker_name_cleaned[3:]  # Remove leading "de"

        except Exception as e:
            print(f"An error occurred: {e}")
    
        # Return the cleaned broker name
        return broker_name_cleaned.strip()
    
    # Apply the clean_broker function to the 'Broker' column and remove the "ClasificadosOnline" text
    df["broker"] = df.apply(lambda row: clean_broker(row["broker"], row["barrio"]), axis=1)
    

    # Extract the number of bedrooms and bathrooms from the 'Rooms' column
    # Extract bedrooms (including cases with >=)
    df['bedrooms'] = df['rooms'].str.extract(r'(>=\s*\d+|\d+)\s+Cuartos', expand=False)

    # Extract bathrooms
    df['bathrooms'] = df['rooms'].str.extract(r'(>=\s*\d+|\d+)\s+Baños', expand=False)

    # Fill NaN values with string 0
    df[['bedrooms', 'bathrooms']] = df[['bedrooms', 'bathrooms']].fillna("0")


    # Drop the Rooms column
    df.drop(columns=['rooms'], inplace=True)

    # Fill missing values in the 'barrio' and 'pueblo' columns with empty strings
    df['barrio'] = df['barrio'].fillna('')
    df['pueblo'] = df['pueblo'].fillna('')

    # Strip any leading or trailing whitespace from the 'barrio' and 'pueblo' columns
    df['barrio'] = df['barrio'].str.strip()
    df['pueblo'] = df['pueblo'].str.strip()

    # Load the municipio data from the CSV file
    municipio = pd.read_csv('PR_Municipios')

    # Check if the word from the name column in municipio is present in the pueblo column within data
    df['pueblo_name'] = df['pueblo'].apply(lambda x: next((name for name in municipio['name'] if name in x), None))

    # Merge the dataframes on the extracted pueblo names and the name column
    merged_df = pd.merge(df, municipio, left_on='pueblo_name', right_on='name', how='left')

    # Add the region column to the original data dataframe
    df['region'] = merged_df['region']

    # Drop the temporary pueblo_name column
    df.drop(columns=['pueblo_name'], inplace=True)

    # Clean the 'price' column by removing the dollar sign and commas and converting it to a float
    df['price'] = df['price'].str.replace('$','').str.replace(',','').astype(float)

    # Remove duplicate rows based on the 'PropertyID' column
    # Keep the first occurrence and log the duplicates
    duplicates = df[df.duplicated(subset=['propertyid'], keep=False)]
    
    if not duplicates.empty:
        num_duplicates = duplicates['propertyid'].nunique()  # Count unique duplicate property IDs
        print(f"Duplicate Properties Found: {num_duplicates}")
        print(duplicates.to_string(index=False))

    else:
        print("No duplicate properties found")

    # Drop duplicates and keep the first occurrence
    df = df.drop_duplicates(subset=['propertyid'], keep='first').reset_index(drop=True)

    return df

In [45]:
# Initialize an empty list to store all data
all_cards = []

# Initialize a page counter (optional, for debugging or tracking pages)
page_number = 1

# Infinite loop to navigate through pages
for x in range(3):

    dropdown_element = driver.find_element(By.XPATH, '//*[@id="jumpMenu"]')
    select = Select(dropdown_element)
    selected_option = select.first_selected_option

    print(f"Scraping page {page_number}...")

    if selected_option.text !=  'Ultimos Publicados (Más Recientes Primeros)':
        dropdown_element.click()
        time.sleep(2)
        ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
        ultimos_publicados.click()
        time.sleep(7)


    # Extract data from the current page
    list_cards = []

    for i in range(1, 16):  # Adjust the range as per the number of rows on a page
        dictionary_cards = {}
        dictionary_cards_2 = {}
        
        # Table 1 Data
    
        table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
        table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_barrio = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td > a:nth-child(1)'
        table_1_pueblo = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td > a:nth-child(3)'
        table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        table_1_piclink = table_1 + f'> tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td > div > a > img'
        
        try:
            dictionary_cards['title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
            dictionary_cards['rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
            dictionary_cards['price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
            
            # type validation, since a property can have two types
            try: 

                table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
                dictionary_cards['type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text

            except NoSuchElementException:
                dictionary_cards['type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text


            dictionary_cards['barrio'] = driver.find_element(By.CSS_SELECTOR, table_1_barrio).text
            dictionary_cards['pueblo'] = driver.find_element(By.CSS_SELECTOR, table_1_pueblo).text
            link_1 = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
            dictionary_cards['link'] = link_1
            dictionary_cards['propertyid'] = extract_property_id(link_1)

            if "MultipleSellers" in dictionary_cards['link']:
                dictionary_cards['broker'] = 'Multiple Sellers'

            else:

                # not all properties have brokers
                try:
                    dictionary_cards['broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards['broker'] = 'No Broker'


            dictionary_cards['piclink'] = driver.find_element(By.CSS_SELECTOR, table_1_piclink).get_attribute('src')

            list_cards.append(dictionary_cards)
            
        except NoSuchElementException:
            pass

        # Table 2 Data (similar logic for table_2)
        table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
        table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_barrio = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td > a:nth-child(1)'
        table_2_pueblo = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td > a:nth-child(3)'
        table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        table_2_piclink = table_2 + f'> tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td > div > a > img'

        try:
            dictionary_cards_2['title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
            dictionary_cards_2['rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
            dictionary_cards_2['price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
            
            # type validation, since a property can have two types
            try:
                table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
                dictionary_cards_2['type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text
            except NoSuchElementException:
                dictionary_cards_2['type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
            
            dictionary_cards_2['barrio'] = driver.find_element(By.CSS_SELECTOR, table_2_barrio).text
            dictionary_cards_2['pueblo'] = driver.find_element(By.CSS_SELECTOR, table_2_pueblo).text
            link_2 = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
            dictionary_cards_2['link'] = link_2
            dictionary_cards_2['propertyid'] = extract_property_id(link_2)

            if "MultipleSellers" in dictionary_cards_2['link']:
                dictionary_cards_2['broker'] = 'Multiple Sellers'

            else:
                try:
                    dictionary_cards_2['broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards_2['broker'] = 'No Broker'

            dictionary_cards_2['piclink'] = driver.find_element(By.CSS_SELECTOR, table_2_piclink).get_attribute('src')

            list_cards.append(dictionary_cards_2)
            
        except NoSuchElementException:
            pass

    # Append the current page's data to the main list
    all_cards.extend(list_cards)

    # Try to click the "Next" button
    try:
        time.sleep(10)
        next_page = driver.find_element(By.XPATH, '//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
        next_page.click()
        time.sleep(7)  # Add delay to allow page to load
        page_number += 1
    except NoSuchElementException:
        print("No more pages available.")
        break

# Create a single DataFrame from all collected data
df = pd.DataFrame(all_cards)
cleaned_df = clean_real_estate_data(df)

Scraping page 1...
Scraping page 2...
Scraping page 3...
Duplicate Properties Found:
                                   title     price   type           barrio  \
29  BO. CARRAIZO, SOLAR EN TRUJILLO ALTO  100000.0  Solar  Barrio-Carraizo   
30  BO. CARRAIZO, SOLAR EN TRUJILLO ALTO  100000.0  Solar  Barrio-Carraizo   

           pueblo                                               link  \
29  Trujillo Alto  https://www.clasificadosonline.com/UDRealEstat...   
30  Trujillo Alto  https://www.clasificadosonline.com/UDRealEstat...   

   propertyid                   broker  \
29    4838700  KW GRAND HOMES CAROLINA   
30    4838700  KW GRAND HOMES CAROLINA   

                                              piclink bedrooms bathrooms  \
29  https://imgcache.clasificadosonline.com//PP/FS...        0         0   
30  https://imgcache.clasificadosonline.com//PP/FS...        0         0   

    region  
29   metro  
30   metro  


In [30]:
cleaned_df.to_csv("classifieds_data_v5.csv", index=False)

In [2]:
data = pd.read_csv('classifieds_data_v3.csv')

In [ ]:
cleaned_df['propertyid'].unique()

In [ ]:
cleaned_df[cleaned_df['broker'] == 'No Broker']

In [46]:
cleaned_df[cleaned_df.duplicated(subset=['propertyid'], keep=False)]

,title,price,type,barrio,pueblo,link,propertyid,broker,piclink,bedrooms,bathrooms,region


In [47]:
cleaned_df

,title,price,type,barrio,pueblo,link,propertyid,broker,piclink,bedrooms,bathrooms,region
0,Dominguito Carr 651,200000.0,Finca,Barrio-Dominguito,Arecibo,https://www.clasificadosonline.com/UDRealEstat...,4833822,AG QUALITY PROPERTIES,https://imgcache.clasificadosonline.com//PP/FS...,0,0,north
1,Home For Sale at Dorado Beach East!,3500000.0,Casa,Urbanizacion-Dorado Beach East,Dorado,https://www.clasificadosonline.com/UDRealEstat...,4781400,AIDA L. COLON-UNIQUE PROPERTIES REALTY,https://imgcache.clasificadosonline.com//PP/FS...,4,4,north
2,STUNNING 2-STORY LUXURY PENTHOUSE,2995000.0,Apartamento,Urbanizacion-Roosevelt,San Juan - Hato Rey,https://www.clasificadosonline.com/UDRealEstat...,4810589,No Broker,https://imgcache.clasificadosonline.com//FF/FS...,3,0,metro
3,Hatillo-Bo. Pajuil,130000.0,Casa,Barrio-Pajuil,Hatillo,https://www.clasificadosonline.com/UDRealEstat...,4838111,Angels Masters Realty,https://imgcache.clasificadosonline.com//PP/FS...,2,1,north
4,INVIERTE | LOCAL COMERCIAL + APTO 3/1,110000.0,Comercial,Barrio-Pueblo,Toa Baja,https://www.clasificadosonline.com/UDRealEstat...,4838702,No Broker,https://imgcache.clasificadosonline.com//FF/FS...,3,4,metro
...,...,...,...,...,...,...,...,...,...,...,...,...
84,Urb. EntreRios Trujillo Alto,549000.0,Casa,"Urbanizacion-Entrerios, Encantada",Trujillo Alto,https://www.clasificadosonline.com/UDRealEstat...,4837390,BLACKTIE,https://imgcache.clasificadosonline.com//PP/FS...,4,3,metro
85,"Terreno, Estancias de Cupey, 1,036 mc",70000.0,Solar,Urbanizacion-Estancias De Cupey,San Juan - Río Piedras,https://www.clasificadosonline.com/UDRealEstat...,4829334,Adalberto Navarro,https://imgcache.clasificadosonline.com//PP/FS...,0,0,metro
86,Ayudas de Vivienda Federal,118000.0,Casa Repo,Urbanizacion-Miraflores,Bayamón,https://www.clasificadosonline.com/UDRealEstat...,4838696,Juan Then Realty,https://imgcache.clasificadosonline.com//PP/FS...,3,1,metro
87,BRISTOL- terraza unica beach front,4975000.0,Apartamento,Condominio-Bristol,San Juan - Condado-Miramar,https://www.clasificadosonline.com/UDRealEstat...,4836063,MiCORREDOR.com,https://imgcache.clasificadosonline.com//PP/FS...,3,3,metro


In [51]:


# Supabase credentials
SUPABASE_URL = "https://ejowipegnoutzdebhndu.supabase.co"  # Replace with your Supabase project URL
SUPABASE_KEY = ""  # Replace with your Supabase service role key

# Initialize Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Function to upload properties to the database
def upload_properties_to_database(df):
    today = datetime.now().date()
    
    for _,property in df.iterrows():
        property_id = property["propertyid"]

        # Check if the property already exists
        existing_property = supabase.table("properties").select("*").eq("property_id", property_id).execute()

        if existing_property.data:
            # Update existing property
            supabase.table("properties").update({
                "last_seen": str(today),
                "times_seen": existing_property.data[0]["times_seen"] + 1
            }).eq("property_id", property_id).execute()
        else:
            # Insert new property
            supabase.table("properties").insert({
                "property_id": property_id,
                "title": property["title"],
                "price": property["price"],
                "type": property["type"],
                "barrio": property["barrio"],
                "pueblo": property["pueblo"],
                "link": property["link"],
                "broker": property["broker"],
                "piclink": property["piclink"],
                "bedrooms": property["bedrooms"],
                "bathrooms": property["bathrooms"],
                "region": property["region"],
                "first_seen": str(today),
                "last_seen": str(today),
                "times_seen": 1
            }).execute()




In [53]:
# Upload properties to the database
upload_properties_to_database(cleaned_df)